In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [2]:
!pip install haversine
import haversine as hs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Import all relevant libraries
from sklearn.preprocessing import StandardScaler

In [4]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
x_train = pd.read_csv("/content/drive/MyDrive/x_train.csv", header=0).iloc[:, 1:]
x_test = pd.read_csv("/content/drive/MyDrive/x_test.csv", header=0).iloc[:, 1:]
y_train = pd.read_csv("/content/drive/MyDrive/y_train.csv", header=0).iloc[:, 1:]
y_test = pd.read_csv("/content/drive/MyDrive/y_test.csv", header=0).iloc[:, 1:]

In [6]:
pd.set_option("display.max_columns", None)
x_train.head(3)

,latitude,longitude,accommodates,bathrooms_text,beds,minimum_nights,maximum_nights,maximum_minimum_nights,minimum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,reviews_per_month,Dist_Acropolis,Dist_Syntagma,neighbourhood_cleansed_ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,neighbourhood_cleansed_ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,neighbourhood_cleansed_ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,neighbourhood_cleansed_ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,neighbourhood_cleansed_ΑΚΡΟΠΟΛΗ,neighbourhood_cleansed_ΑΜΠΕΛΟΚΗΠΟΙ,neighbourhood_cleansed_ΑΝΩ ΚΥΨΕΛΗ,neighbourhood_cleansed_ΑΝΩ ΠΑΤΗΣΙΑ,neighbourhood_cleansed_ΒΟΤΑΝΙΚΟΣ,neighbourhood_cleansed_ΓΚΑΖΙ,neighbourhood_cleansed_ΓΚΥΖΗ,neighbourhood_cleansed_ΓΟΥΒΑ,neighbourhood_cleansed_ΓΟΥΔΙ,neighbourhood_cleansed_ΕΛΛΗΝΟΡΩΣΩΝ,neighbourhood_cleansed_ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,neighbourhood_cleansed_ΖΑΠΠΕΙΟ,neighbourhood_cleansed_ΘΗΣΕΙΟ,neighbourhood_cleansed_ΙΛΙΣΙΑ,neighbourhood_cleansed_ΚΕΡΑΜΕΙΚΟΣ,neighbourhood_cleansed_ΚΟΛΟΚΥΝΘΟΥ,neighbourhood_cleansed_ΚΟΛΩΝΑΚΙ,neighbourhood_cleansed_ΚΟΛΩΝΟΣ,neighbourhood_cleansed_ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ,neighbourhood_cleansed_ΚΥΨΕΛΗ,neighbourhood_cleansed_ΛΥΚΑΒΗΤΤΟΣ,neighbourhood_cleansed_ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ,neighbourhood_cleansed_ΝΕΑ ΚΥΨΕΛΗ,neighbourhood_cleansed_ΝΕΟΣ ΚΟΣΜΟΣ,neighbourhood_cleansed_ΝΙΡΒΑΝΑ,neighbourhood_cleansed_ΠΑΓΚΡΑΤΙ,neighbourhood_cleansed_ΠΑΤΗΣΙΑ,neighbourhood_cleansed_ΠΕΔΙΟ ΑΡΕΩΣ,neighbourhood_cleansed_ΠΕΝΤΑΓΩΝΟ,neighbourhood_cleansed_ΠΕΤΡΑΛΩΝΑ,neighbourhood_cleansed_ΠΛΑΤΕΙΑ ΑΜΕΡΙΚΗΣ,neighbourhood_cleansed_ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ,neighbourhood_cleansed_ΠΟΛΥΓΩΝΟ,neighbourhood_cleansed_ΠΡΟΜΠΟΝΑ,neighbourhood_cleansed_ΡΗΓΙΛΛΗΣ,neighbourhood_cleansed_ΡΙΖΟΥΠΟΛΗ,neighbourhood_cleansed_ΣΕΠΟΛΙΑ,neighbourhood_cleansed_ΣΤΑΔΙΟ,neighbourhood_cleansed_ΣΤΑΘΜΟΣ ΛΑΡΙΣΗΣ,room_type_Hotel room,room_type_Private room,room_type_Shared room,host_is_superhost_1.0,host_has_profile_pic_1.0,host_identity_verified_1.0,has_availability_1,instant_bookable_1,has_license_1
0,37.98774,23.72788,5,1.0,4.0,2,31,2,31,2.0,31.0,29,59,89,179,0,1.20,1.697690,1.466880,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1,37.99185,23.74902,6,1.0,2.0,1,29,1,1125,1.0,1125.0,0,0,0,62,75,1.26,2.953900,2.177478,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
2,37.98972,23.73995,3,1.0,1.0,1,1125,1,1125,1.0,1125.0,29,59,89,90,36,0.57,2.273573,1.615544,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [7]:
x_test.head(3)

,latitude,longitude,accommodates,bathrooms_text,beds,minimum_nights,maximum_nights,maximum_minimum_nights,minimum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,reviews_per_month,Dist_Acropolis,Dist_Syntagma,neighbourhood_cleansed_ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,neighbourhood_cleansed_ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,neighbourhood_cleansed_ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,neighbourhood_cleansed_ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,neighbourhood_cleansed_ΑΚΡΟΠΟΛΗ,neighbourhood_cleansed_ΑΜΠΕΛΟΚΗΠΟΙ,neighbourhood_cleansed_ΑΝΩ ΚΥΨΕΛΗ,neighbourhood_cleansed_ΑΝΩ ΠΑΤΗΣΙΑ,neighbourhood_cleansed_ΒΟΤΑΝΙΚΟΣ,neighbourhood_cleansed_ΓΚΑΖΙ,neighbourhood_cleansed_ΓΚΥΖΗ,neighbourhood_cleansed_ΓΟΥΒΑ,neighbourhood_cleansed_ΓΟΥΔΙ,neighbourhood_cleansed_ΕΛΛΗΝΟΡΩΣΩΝ,neighbourhood_cleansed_ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,neighbourhood_cleansed_ΖΑΠΠΕΙΟ,neighbourhood_cleansed_ΘΗΣΕΙΟ,neighbourhood_cleansed_ΙΛΙΣΙΑ,neighbourhood_cleansed_ΚΕΡΑΜΕΙΚΟΣ,neighbourhood_cleansed_ΚΟΛΟΚΥΝΘΟΥ,neighbourhood_cleansed_ΚΟΛΩΝΑΚΙ,neighbourhood_cleansed_ΚΟΛΩΝΟΣ,neighbourhood_cleansed_ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ,neighbourhood_cleansed_ΚΥΨΕΛΗ,neighbourhood_cleansed_ΛΥΚΑΒΗΤΤΟΣ,neighbourhood_cleansed_ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ,neighbourhood_cleansed_ΝΕΑ ΚΥΨΕΛΗ,neighbourhood_cleansed_ΝΕΟΣ ΚΟΣΜΟΣ,neighbourhood_cleansed_ΝΙΡΒΑΝΑ,neighbourhood_cleansed_ΠΑΓΚΡΑΤΙ,neighbourhood_cleansed_ΠΑΤΗΣΙΑ,neighbourhood_cleansed_ΠΕΔΙΟ ΑΡΕΩΣ,neighbourhood_cleansed_ΠΕΝΤΑΓΩΝΟ,neighbourhood_cleansed_ΠΕΤΡΑΛΩΝΑ,neighbourhood_cleansed_ΠΛΑΤΕΙΑ ΑΜΕΡΙΚΗΣ,neighbourhood_cleansed_ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ,neighbourhood_cleansed_ΠΟΛΥΓΩΝΟ,neighbourhood_cleansed_ΠΡΟΜΠΟΝΑ,neighbourhood_cleansed_ΡΗΓΙΛΛΗΣ,neighbourhood_cleansed_ΡΙΖΟΥΠΟΛΗ,neighbourhood_cleansed_ΣΕΠΟΛΙΑ,neighbourhood_cleansed_ΣΤΑΔΙΟ,neighbourhood_cleansed_ΣΤΑΘΜΟΣ ΛΑΡΙΣΗΣ,room_type_Hotel room,room_type_Private room,room_type_Shared room,host_is_superhost_1.0,host_has_profile_pic_1.0,host_identity_verified_1.0,has_availability_1,instant_bookable_1,has_license_1
0,37.96859,23.75275,2,1.0,1.0,2,32,2,1125,2.0,1125.0,17,47,77,258,3,0.92,2.398668,1.758095,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0
1,37.98599,23.71586,4,1.0,3.0,2,1125,2,1125,2.0,1125.0,0,0,0,0,32,0.32,1.731598,2.017828,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
2,37.99410,23.74336,2,1.0,1.0,3,1125,3,1125,3.0,1125.0,0,0,0,54,9,0.22,2.844840,2.171475,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1


In [8]:
y_train.head(3)

,log_price
0,4.094345
1,3.135494
2,4.248495


In [9]:
y_test.head(3)

,log_price
0,3.401197
1,5.010635
2,3.367296


In [10]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((7185, 71), (1797, 71), (7185, 1), (1797, 1))



### Message passing train set

In [11]:
lat = x_train.latitude.to_list()
lon = x_train.longitude.to_list()

In [12]:
from tqdm import tqdm

In [13]:
distances = []
n = len(lat)
for i in tqdm(range(n)):
    row = []
    for x, y in zip(lat, lon):

        dist = hs.haversine((x, y), (lat[i], lon[i]))
        row.append(dist)

    distances.append(row)

100%|██████████| 7185/7185 [03:31<00:00, 33.98it/s]


In [17]:
df_new = []
for index, row in tqdm(enumerate(distances)):
    res = sorted(range(len(row)), key=lambda sub: row[sub])[:3]

    msg = []
    for pos in res:
        msg.append(x_train.iloc[pos])

    temp1 = [0.1 * x + 0.05 * y for x, y in zip(msg[0], msg[1])]
    temp2 = [x + 0.05 * y for x, y in zip(temp1, msg[2])]

    result = [0.8 * x + y for x, y in zip(x_train.iloc[index].to_list(), temp2)]

    df_new.append(result)

df = pd.DataFrame(df_new)

7185it [00:25, 279.97it/s]


### Message passing test set


In [19]:
lat_test = x_test.latitude.to_list()
lon_test = x_test.longitude.to_list()

In [20]:
distances_test = []
n = len(lat_test)
for i in tqdm(range(n)):
    row = []
    for x, y in zip(lat, lon):

        dist = hs.haversine((x, y), (lat_test[i], lon_test[i]))
        row.append(dist)

    distances_test.append(row)

100%|██████████| 1797/1797 [01:02<00:00, 28.96it/s]


In [21]:
df_new = []
for index, row in tqdm(enumerate(distances_test)):
    res = sorted(range(len(row)), key=lambda sub: row[sub])[:3]

    msg = []
    for pos in res:
        msg.append(x_train.iloc[pos])

    temp1 = [0.1 * x + 0.05 * y for x, y in zip(msg[0], msg[1])]
    temp2 = [x + 0.05 * y for x, y in zip(temp1, msg[2])]

    result = [0.8 * x + y for x, y in zip(x_test.iloc[index].to_list(), temp2)]

    df_new.append(result)

df_test = pd.DataFrame(df_new)

1797it [00:09, 195.46it/s]


In [23]:
x_train = df
x_test = df_test

# Machine Learning Models

## MLP

In [27]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [28]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train))
x_test = pd.DataFrame(scaler.transform(x_test))

In [29]:
seed = 42
grid = {
    "hidden_layer_sizes": [(128, 128), (128, 256, 256), (256, 256)],
    "solver": ["adam"],
    "learning_rate_init": [0.001, 0.0001],
    "activation": ["relu"],
    "alpha": [0.0001, 0.001, 0.005],
    "early_stopping": [True, False],
    "max_iter": [300],
    "random_state": [seed],
}

In [30]:
mlp = GridSearchCV(
    estimator=MLPRegressor(), param_grid=grid, cv=4, n_jobs=-1, verbose=2
)
mlp.fit(x_train, y_train)

Fitting 4 folds for each of 36 candidates, totalling 144 fits


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=4, estimator=MLPRegressor(), n_jobs=-1,
             param_grid={'activation': ['relu'],
                         'alpha': [0.0001, 0.001, 0.005],
                         'early_stopping': [True, False],
                         'hidden_layer_sizes': [(128, 128), (128, 256, 256),
                                                (256, 256)],
                         'learning_rate_init': [0.001, 0.0001],
                         'max_iter': [300], 'random_state': [42],
                         'solver': ['adam']},
             verbose=2)

In [31]:
mlp_best_params = mlp.best_params_
mlp_best_params

{'activation': 'relu',
 'alpha': 0.005,
 'early_stopping': True,
 'hidden_layer_sizes': (256, 256),
 'learning_rate_init': 0.001,
 'max_iter': 300,
 'random_state': 42,
 'solver': 'adam'}

In [32]:
best_mlp = MLPRegressor(
    hidden_layer_sizes=mlp_best_params["hidden_layer_sizes"],
    solver=mlp_best_params["solver"],
    learning_rate_init=mlp_best_params["learning_rate_init"],
    activation=mlp_best_params["activation"],
    alpha=mlp_best_params["alpha"],
    early_stopping=mlp_best_params["early_stopping"],
    max_iter=mlp_best_params["max_iter"],
    random_state=mlp_best_params["random_state"],
).fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [33]:
pred_mlp = best_mlp.predict(x_test)
# Calculate the Mean absolute Error using the mean_absolute_error function.
print("Test Data")
print(
    "Mean Absolute Error : %0.3f"
    % mean_absolute_error(y_test.pow(2.7183), np.power(pred_mlp, 2.7183))
)
print(
    "Mean Absolute Percentage Error : %0.3f"
    % mean_absolute_percentage_error(y_test.pow(2.7183), np.power(pred_mlp, 2.7183))
)

Test Data
Mean Absolute Error : 10.659
Mean Absolute Percentage Error : 0.236
